In [1]:
from evaluation.EvalRSRunner import ChallengeDataset

import gc
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
dataset = ChallengeDataset()

LFM dataset already downloaded. Skipping download.
Loading dataset.
Generating Train/Test Split.
Generating dataset hashes.


In [3]:
train, test = dataset.get_sample_train_test()
train.shape, test.shape

((6869679, 6), (29722, 3))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6869679 entries, 0 to 37926426
Data columns (total 6 columns):
 #   Column            Dtype
---  ------            -----
 0   user_id           int64
 1   artist_id         int64
 2   album_id          int64
 3   track_id          int64
 4   timestamp         int64
 5   user_track_count  int64
dtypes: int64(6)
memory usage: 366.9 MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29722 entries, 37926134 to 211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   29722 non-null  int64
 1   track_id  29722 non-null  int64
 2   fold      29722 non-null  int64
dtypes: int64(3)
memory usage: 928.8 KB


In [6]:
df_tracks = dataset.df_tracks

In [7]:
df_tracks.shape

(820998, 12)

In [8]:
df_tracks.head()

,track,artist_id,artist,albums_id,albums,urlSong,arousal_predicted,valence_predicted,lastfm_id,has_emotion_tags,has_social_tags,isClassic
track_id,,,,,,,,,,,,
1,A Matter of Time,3,Foo Fighters,"[1, 67655, 605875, 682938, 889936, 2797722, 34...","['Wasting Light', 'Wasting Light (Deluxe Versi...",http://lyrics.wikia.com/Foo_Fighters:A_Matter_...,-0.066624,-0.256498,None,None,None,False
2,Hangar 18,1,Megadeth,"[2, 693, 821, 2113, 12071, 13309, 17004, 83531...","['Rust In Peace', 'Countdown To Extinction', '...",http://lyrics.wikia.com/Megadeth:Hangar_18,-0.024426,-0.441335,None,None,None,False
3,Up the Downstair,4,Porcupine Tree,"[84, 116, 13570, 283314, 302086, 303154, 35956...","['Up the Downstair', 'Coma Divine', 'Up The Do...",http://lyrics.wikia.com/Porcupine_Tree:Up_The_...,-0.324860,-0.283030,TRSHLAP128F424880E,True,True,False
5,Mixtaped,5,No-Man,"[5, 7654681]","['Schoolyard Ghosts', 'Schoolyard Ghosts Disc 1']",http://lyrics.wikia.com/No-Man:Mixtaped,0.009914,-0.708832,None,None,None,False
6,Sounds that I Hear,6,Airbag,"[6, 340, 1139848]","['Sounds That I Hear', 'Identity', nan]",None,NaN,NaN,None,None,None,None


In [9]:
df = train[['user_id', 'track_id', 'timestamp', 'user_track_count']].sort_values('timestamp')
df = pd.DataFrame(df).join(dataset.df_users, on='user_id', how='left')

In [10]:
df.head()

,user_id,track_id,timestamp,user_track_count,country,age,gender,playcount,registered_unixtime,country_id,...,relative_le_per_hour14,relative_le_per_hour15,relative_le_per_hour16,relative_le_per_hour17,relative_le_per_hour18,relative_le_per_hour19,relative_le_per_hour20,relative_le_per_hour21,relative_le_per_hour22,relative_le_per_hour23
37508031,2405701,1147113,1113164704,1,None,-1,None,230,1113151438,-1,...,0.0296,0.0329,0.0197,NaN,0.0428,0.0461,0.0691,0.0329,0.1283,0.0921
37508030,2405701,10722,1113164995,2,None,-1,None,230,1113151438,-1,...,0.0296,0.0329,0.0197,NaN,0.0428,0.0461,0.0691,0.0329,0.1283,0.0921
37508029,2405701,171719,1113165276,3,None,-1,None,230,1113151438,-1,...,0.0296,0.0329,0.0197,NaN,0.0428,0.0461,0.0691,0.0329,0.1283,0.0921
37508028,2405701,93031,1113166830,2,None,-1,None,230,1113151438,-1,...,0.0296,0.0329,0.0197,NaN,0.0428,0.0461,0.0691,0.0329,0.1283,0.0921
37508027,2405701,12590028,1113167111,1,None,-1,None,230,1113151438,-1,...,0.0296,0.0329,0.0197,NaN,0.0428,0.0461,0.0691,0.0329,0.1283,0.0921


playcount data sections

In [11]:
# playcount seperation
pc_1 = df[df['playcount'] <= 10].groupby(['user_id'], sort=False)['track_id'].agg(list)
pc_2 = df[(10 < df['playcount']) & (df['playcount'] <= 100)].groupby(['user_id'], sort=False)['track_id'].agg(list)
pc_3 = df[(100 < df['playcount']) & (df['playcount'] <= 1000)].groupby(['user_id'], sort=False)['track_id'].agg(list)
pc_4 = df[1000 < df['playcount']].groupby(['user_id'], sort=False)['track_id'].agg(list)


# gender seperation
p_m = df[df['gender'] == 'm'].groupby(['user_id'], sort=False)['track_id'].agg(list)
p_f = df[df['gender'] == 'f'].groupby(['user_id'], sort=False)['track_id'].agg(list)
p_n = df[(df['gender'] != 'm') & (df['gender'] != 'f')].groupby(['user_id'], sort=False)['track_id'].agg(list)


# user_track_count seperation
df_trackid = df.groupby(['user_id'], sort=False)['track_id'].agg(list)
df = pd.DataFrame(df_trackid).join(df.groupby('user_id', as_index=True, sort=False)[['user_track_count']].sum(), on='user_id', how='left')
df = pd.DataFrame(df).join(dataset.df_users, on='user_id', how='left')

tc_1 = df[df['user_track_count'] <= 100]['track_id']
tc_2 = df[(100 < df['user_track_count']) & (df['user_track_count'] <= 1000)]['track_id']
tc_3 = df[1000 < df['user_track_count']]['track_id']

### Prepare data for BILSTM model

In [27]:
# Assuming df_tracks has track_id as its index
unique_track_ids = df_tracks.index.unique()
track_id_to_token = {track_id: i for i, track_id in enumerate(unique_track_ids)}
token_to_track_id = {i: track_id for track_id, i in track_id_to_token.items()}

# Update vocab_size to reflect the number of unique tracks
vocab_size = len(unique_track_ids)


In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

sequences = list(pc_1.tolist())  # Assuming pc_1 is your series of track ID lists

# Step 2: Create input-target pairs
X, y = [], []

for sequence in sequences:
    for i in range(1, len(sequence)):
        X.append(sequence[:i])
        y.append(sequence[i])

# Step 3: Determine the max sequence length for padding; you might adjust this based on your model's capacity
max_sequence_length = max(len(x) for x in X)
vocab_size = len(df_tracks)

# Step 4: Pad sequences
X_padded = pad_sequences(X, maxlen=max_sequence_length)

# Convert y to numpy array for compatibility with sklearn functions
y = np.array(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [30]:
# Function to update sequences with tokens
def update_sequences_with_tokens(sequences, mapping):
    return [[mapping.get(track_id, 0) for track_id in sequence] for sequence in sequences]

# Update X_train and X_val sequences
X_train_tokens = update_sequences_with_tokens(X_train, track_id_to_token)
X_val_tokens = update_sequences_with_tokens(X_val, track_id_to_token)

# For y_train and y_val, you can directly apply the mapping since they're single IDs and not sequences
y_train_tokens = np.array([track_id_to_token[track_id] for track_id in y_train])
y_val_tokens = np.array([track_id_to_token[track_id] for track_id in y_val])

# Don't forget to pad the sequences again if necessary
X_train_padded = pad_sequences(X_train_tokens, maxlen=max_sequence_length)
X_val_padded = pad_sequences(X_val_tokens, maxlen=max_sequence_length)


### Define BILSTM Model

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_shape=(max_sequence_length,)),
    Dropout(0.2),  # Dropout for input layer
    Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)),  # Adding dropout to LSTM
    Dense(vocab_size, activation='softmax')
])

# Customizing the learning rate
optimizer = Adam(learning_rate=0.001)  # You might want to experiment with different rates

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 403, 100)       │    82,099,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 403, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 256)            │       234,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 820998)         │   210,996,486 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 293,330,782 (1.09 GB)

 Trainable params: 293,330,782 (1.09 GB)

 Non-trainable params: 0 (0.00 B)

In [67]:
# Train the model
history = model.fit(
    X_train_padded, y_train_tokens,
    batch_size=32,  # You can adjust the batch size
    epochs=1,      # And the number of epochs according to your dataset and how the training progresses
    validation_data=(X_val_padded, y_val_tokens)
)


57/57 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.0000e+00 - loss: 13.4183 - val_accuracy: 0.0000e+00 - val_loss: 14.6505


In [68]:
history.history

{'accuracy': [0.0],
 'loss': [13.319997787475586],
 'val_accuracy': [0.0],
 'val_loss': [14.650463104248047]}

In [57]:
train_loss, train_acc = model.evaluate(X_train_padded, y_train_tokens, verbose=0)
val_loss, val_acc = model.evaluate(X_val_padded, y_val_tokens, verbose=0)
print(f"Training Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}")

Training Accuracy: 0.0240, Validation Accuracy: 0.0000


In [72]:
def prepare_input_sequence(sequence, track_id_to_token, max_sequence_length):
    # Map track IDs to tokens
    token_sequence = [track_id_to_token.get(track_id, 0) for track_id in sequence]
    # Pad the sequence
    padded_sequence = pad_sequences([token_sequence], maxlen=max_sequence_length, padding='pre')
    return padded_sequence

def predict_next_track(model, sequence, token_to_track_id):
    # Assuming `sequence` is already prepared and padded
    predictions = model.predict(sequence)
    # Get the token with the highest probability
    next_token = np.argmax(predictions, axis=1)[0]
    # Map the token back to the original track ID
    next_track_id = token_to_track_id[next_token]
    return next_track_id

def predict_next_n_tracks(model, sequence, token_to_track_id, n=5):
    # Initial sequence to start predictions
    current_sequence = sequence.copy()
    
    # Store the predicted track IDs
    predicted_tracks = []
    
    for _ in range(n):
        # Prepare the sequence for model input
        prepared_sequence = pad_sequences([current_sequence], maxlen=max_sequence_length)
        
        # Predict the next track
        predictions = model.predict(prepared_sequence, verbose=0)
        next_token = np.argmax(predictions, axis=1)[0]
        next_track_id = token_to_track_id[next_token]
        
        # Append the predicted track ID to the list of predicted tracks
        predicted_tracks.append(next_track_id)
        
        # Update the sequence with the predicted track ID
        current_sequence.append(next_token)
        
        # Optionally, remove the first token to maintain sequence length
        current_sequence = current_sequence[1:]
    
    return predicted_tracks


In [73]:
# Example sequence of track IDs
starting_sequence = pc_1.iloc[5]
n_predictions=5

# Predict the next track ID
predicted_track_ids = predict_next_n_tracks(model, starting_sequence, token_to_track_id, n=n_predictions)


print(f"Next Track IDs: {predicted_track_ids}")


Next Track IDs: [24841, 24841, 24841, 24841, 24841]


In [12]:
def calculate_distinct_tracks(playcount_groups):
    """
    Calculate the number of distinct track IDs in each playcount group.

    :param playcount_groups: Dictionary with playcount group name as key and list of track ID lists as value
    :return: Dictionary with playcount group name as key and number of distinct tracks as value
    """
    distinct_tracks = {}
    for group_name, track_lists in playcount_groups.items():
        # Flatten the list of lists and calculate the number of unique elements
        distinct_tracks[group_name] = len(set([track_id for sublist in track_lists for track_id in sublist]))
    return distinct_tracks

groups = {
    "pc_1": pc_1,
    "pc_2": pc_2,
    "pc_3": pc_3,
    "pc_4": pc_4,
    "tc_1": tc_1,
    "tc_2": tc_2,
    "tc_3": tc_3,
    "p_m": p_m,
    "p_f": p_f,
    "p_n": p_n
}

groups_lists = {
    "pc_1": pc_1.values.tolist(),
    "pc_2": pc_2.values.tolist(),
    "pc_3": pc_3.values.tolist(),
    "pc_4": pc_4.values.tolist(),
    "tc_1": tc_1.values.tolist(),
    "tc_2": tc_2.values.tolist(),
    "tc_3": tc_3.values.tolist(),
    "p_m": p_m.values.tolist(),
    "p_f": p_f.values.tolist(),
    "p_n": p_n.values.tolist()
}

distinct_track_counts = calculate_distinct_tracks(groups_lists)
distinct_track_counts


{'pc_1': 2217,
 'pc_2': 42889,
 'pc_3': 213735,
 'pc_4': 227095,
 'tc_1': 67572,
 'tc_2': 228044,
 'tc_3': 195559,
 'p_m': 222219,
 'p_f': 193310,
 'p_n': 226649}

In [13]:
import pandas as pd 

def calculate_sequence_length_stats(groups):
    """
    Calculate average and standard deviation of sequence lengths within groups.

    :param groups: Dictionary with group name as key and list of track ID lists as value
    :return: Dictionary with group name as key, and a dictionary containing 'avg' and 'std' as value
    """
    results = {}
    for group_name, track_lists in groups.items():
        lengths = [len(track_list) for track_list in track_lists]  # Calculate lengths
        results[group_name] = {
            'avg': pd.Series(lengths).mean(),
            'std': pd.Series(lengths).std()
        }
    return results


In [14]:
for name, group in groups.items():
    print(name)
    print(calculate_sequence_length_stats({name: group}))

pc_1
{'pc_1': {'avg': 205.0, 'std': 136.57452178206591}}
pc_2
{'pc_2': {'avg': 71.34682080924856, 'std': 77.97591296056639}}
pc_3
{'pc_3': {'avg': 211.20023850823938, 'std': 115.20211070225358}}
pc_4
{'pc_4': {'avg': 247.55876057486495, 'std': 98.00893206457755}}
tc_1
{'tc_1': {'avg': 44.939095387371246, 'std': 21.830949933242966}}
tc_2
{'tc_2': {'avg': 235.68387428718432, 'std': 96.2771731300237}}
tc_3
{'tc_3': {'avg': 305.4428708593375, 'std': 83.76659951353588}}
p_m
{'p_m': {'avg': 228.23628185907046, 'std': 98.58360032209987}}
p_f
{'p_f': {'avg': 252.80314351971143, 'std': 100.33596347987756}}
p_n
{'p_n': {'avg': 227.64877494316747, 'std': 114.88051328475704}}


In [15]:
for name, group in groups.items():
    print(name)
    print(len(group))

pc_1
11
pc_2
865
pc_3
9224
pc_4
19622
tc_1
2233
tc_2
23323
tc_3
4166
p_m
10005
p_f
3881
p_n
15836
